<a href="https://colab.research.google.com/github/ZeroLeon/NLP_Project_with_FastAI/blob/master/Question%26AnsweringNLP_Model_with_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QA Model with fastai

In [0]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)
base_dir = '/content/gdrive/My Drive/fastai'
path = Path(base_dir +'/models_process/')
# path.mkdir(parents=True,exist_ok=True) 
os.chdir(path) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd bert_for_SQuAD_en/

/content/gdrive/My Drive/fastai/models_process/bert_for_SQuAD_en


In [0]:
%%capture
!pip install pytorch-transformers

##Week1. Test an English Model for pre-studying

[Starting from this article](https://rsilveira79.github.io/fermenting_gradients/machine_learning/nlp/pytorch/pytorch-transformer-squad/)

In [0]:
import os
import time
import torch
from pytorch_transformers import BertConfig, BertTokenizer, BertForQuestionAnswering

**Docstring of BertForQuestionAnswering:**   

Bert Model with a span classification head on top for extractive question-answering tasks like SQuAD (a linear layers on top of
the hidden-states output to compute `span start logits` and `span end logits`).     The BERT model was proposed in
`BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding`_
by Jacob Devlin, Ming-Wei Chang, Kenton Lee and Kristina Toutanova. It's a bidirectional transformer
pre-trained using a combination of masked language modeling objective and next sentence prediction
on a large corpus comprising the Toronto Book Corpus and Wikipedia.

This model is a PyTorch `torch.nn.Module`_ sub-class. Use it as a regular PyTorch Module and
refer to the PyTorch documentation for all matter related to general usage and behavior.

In [0]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForQuestionAnswering, BertTokenizer)
    # 'xlnet': (XLNetConfig, XLNetForQuestionAnswering, XLNetTokenizer)
}

In [0]:
class QuestionAnswering(object):
    def __init__(self, config_file, weight_file, tokenizer_file, model_type ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.config_class, self.model_class, self.tokenizer_class = MODEL_CLASSES[model_type]
        self.config = self.config_class.from_json_file(config_file)
        self.model = self.model_class(self.config)
        self.model.load_state_dict(torch.load(weight_file, map_location=self.device))
        self.tokenizer = self.tokenizer_class(tokenizer_file)
        self.model_type = model_type
    
    def to_list(self, tensor):
        return tensor.detach().cpu().tolist()

    def get_reply(self, question, passage):
        self.model.eval()
        with torch.no_grad():
            input_ids, _ , tokens = self.prepare_features(question, passage)
            if self.model_type == 'bert':
                span_start,span_end= self.model(input_ids)
                answer = tokens[torch.argmax(span_start):torch.argmax(span_end)+1]
                answer = self.bert_convert_tokens_to_string(answer)
            elif self.model_type == 'xlnet':
                input_vector = {'input_ids': input_ids,
                                'start_positions': None,
                                'end_positions': None }
                outputs = self.model(**input_vector)
                answer = tokens[self.to_list(outputs[1])[0][torch.argmax(outputs[0])]:self.to_list(outputs[3])[0][torch.argmax(outputs[2])]+1]
                answer = self.xlnet_convert_tokens_to_string(answer)
        return answer
    
    def bert_convert_tokens_to_string(self, tokens):
        out_string = ' '.join(tokens).replace(' ##', '').strip()
        if '@' in tokens:
            out_string = out_string.replace(' ', '')
        return out_string

    def xlnet_convert_tokens_to_string(self, tokens):
        out_string = ''.join(tokens).replace('▁', ' ').strip()
        return out_string

    def prepare_features(self, question,  passage, max_seq_length = 300, 
                 zero_pad = False, include_CLS_token = True, include_SEP_token = True):
        ## Tokenzine Input
        tokens_a = self.tokenizer.tokenize(question)
        tokens_b = self.tokenizer.tokenize(passage)
        ## Truncate
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]
        ## Initialize Tokens
        tokens = []
        if include_CLS_token:
            tokens.append(self.tokenizer.cls_token)
        ## Add Tokens and separators
        for token in tokens_a:
            tokens.append(token)
        if include_SEP_token:
            tokens.append(self.tokenizer.sep_token)
        for token in tokens_b:
            tokens.append(token)
        ## Convert Tokens to IDs
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        ## Input Mask 
        input_mask = [1] * len(input_ids)
        ## Zero-pad sequence lenght
        if zero_pad:
            while len(input_ids) < max_seq_length:
                input_ids.append(0)
                input_mask.append(0)
        return torch.tensor(input_ids).unsqueeze(0), input_mask, tokens

###  Instantiate model

In [0]:
bert = QuestionAnswering(
    config_file =   'bert-large-cased-whole-word-masking-finetuned-squad-config.json',
    weight_file=    'bert-large-cased-whole-word-masking-finetuned-squad-pytorch_model.bin',
    tokenizer_file= 'bert-large-cased-whole-word-masking-finetuned-squad-vocab.txt',
    model_type =    'bert'
)

In [0]:
facts = " My wife is great. \
My complete name is Roberto Pereira Silveira. \
I am 40 years old. \
My dog is cool. \
My dog breed is jack russel. \
My dog was born in 2014.\
My dog name is Mallu. \
My dog is 5 years old. \
I am an engineer. \
I was born in 1979. \
My e-mail is rsilveira79@gmail.com."

questions = [
    "What is my complete name?",
    "What is dog name?",
    "What is my dog age?",
    "What is my age?",
    "What is my dog breed?",
    "When I was born?",
    "What is my e-mail?"
]

In [0]:
bert.get_reply(questions[1],facts)

'mallu'

In [0]:
bert.get_reply('What is my dog age?',facts)

'5'

In [0]:
facts2 = 'Martin Luther King Jr. was an American Christian minister and activist \
who became the most visible spokesperson and leader in the Civil Rights Movement from 1955 \
until his assassination in 1968.\
Born in Atlanta, Georgia, King is best known for advancing civil rights through nonviolence \
and civil disobedience, inspired by his Christian beliefs and the nonviolent activism of Mahatma Gandhi.\
King led the 1955 Montgomery bus boycott and in 1957 became the first president \
of the Southern Christian Leadership Conference (SCLC). \
With the SCLC, he led an unsuccessful 1962 struggle against segregation in Albany, Georgia, \
and helped organize the nonviolent 1963 protests in Birmingham, Alabama. \
He helped organize the 1963 March on Washington, where he delivered his famous "I Have a Dream" speech.'

questions2 = [
              'Who is Martin Luther King?',
              'When did King lead the 1955 Motgomery bus boycott?',
              'What beliefs do King have?',
              'Is Martin a great people?'
]





In [0]:
bert.get_reply(questions2[0],facts2)

'an american christian minister and activist'

In [0]:
bert.get_reply(questions2[1],facts2)

'1955'

In [0]:
bert.get_reply(questions2[2],facts2)

'christian'

In [0]:
bert.get_reply(questions2[3],facts2) # model find no answer so it gives a strange reply

'martin luther king jr . was an american christian minister and activist'

### Break into pieces and see what they are doing?

In [0]:
from pytorch_transformers import BertTokenizer,BertForQuestionAnswering

In [0]:
# tokenizer = BertTokenizer.from_pretrained(path_chinese)
# bert_model = BertForQuestionAnswering.from_pretrained(path_chinese)

In [0]:
config_file =   'bert-large-cased-whole-word-masking-finetuned-squad-config.json'
weight_file=    'bert-large-cased-whole-word-masking-finetuned-squad-pytorch_model.bin'
tokenizer_file= 'bert-large-cased-whole-word-masking-finetuned-squad-vocab.txt'
model_type =    'bert'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]
config = config_class.from_json_file(config_file)
model = model_class(config)
model.load_state_dict(torch.load(weight_file, map_location=device))
tokenizer = tokenizer_class(tokenizer_file)

In [0]:
tokens_a = tokenizer.tokenize(questions2[0])
tokens_b = tokenizer.tokenize(facts2)

In [0]:
tokens_a

['who', 'is', 'ma', '##rt', '##in', 'l', '##uth', '##er', 'king', '?']

In [0]:
tokens_b

['ma',
 '##rt',
 '##in',
 'l',
 '##uth',
 '##er',
 'king',
 'j',
 '##r',
 '.',
 'was',
 'an',
 'am',
 '##eric',
 '##an',
 'ch',
 '##rist',
 '##ian',
 'minister',
 'and',
 'activist',
 'who',
 'became',
 'the',
 'most',
 'visible',
 'spokesperson',
 'and',
 'leader',
 'in',
 'the',
 'civil',
 'rights',
 'movement',
 'from',
 '1955',
 'until',
 'his',
 'assassination',
 'in',
 '1968',
 '.',
 'born',
 'in',
 'at',
 '##lant',
 '##a',
 ',',
 'g',
 '##eor',
 '##gia',
 ',',
 'king',
 'is',
 'best',
 'known',
 'for',
 'advancing',
 'civil',
 'rights',
 'through',
 'non',
 '##vio',
 '##len',
 '##ce',
 'and',
 'civil',
 'di',
 '##so',
 '##bedience',
 ',',
 'inspired',
 'by',
 'his',
 'ch',
 '##rist',
 '##ian',
 'beliefs',
 'and',
 'the',
 'non',
 '##vio',
 '##lent',
 'activism',
 'of',
 'ma',
 '##hat',
 '##ma',
 'g',
 '##and',
 '##hi',
 '.',
 'king',
 'led',
 'the',
 '1955',
 'mon',
 '##t',
 '##go',
 '##mer',
 '##y',
 'bus',
 'boycott',
 'and',
 'in',
 '1957',
 'became',
 'the',
 'first',
 'pres

In [0]:
max_seq_length = 300
zero_pad = False
include_CLS_token = True
include_SEP_token = True

In [0]:
if len(tokens_a) > max_seq_length - 2:
  tokens_a = tokens_a[0:(max_seq_length - 2)]
  ## Initialize Tokens
tokens = []
if include_CLS_token:
    tokens.append(tokenizer.cls_token)
## Add Tokens and separators
for token in tokens_a:
    tokens.append(token)
if include_SEP_token:
    tokens.append(tokenizer.sep_token)
for token in tokens_b:
    tokens.append(token)
## Convert Tokens to IDs
input_ids = tokenizer.convert_tokens_to_ids(tokens)
## Zero-pad sequence length
if zero_pad:
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
input_ids=torch.tensor(input_ids).unsqueeze(0)


In [0]:
def bert_convert_tokens_to_string(tokens):
    out_string = ' '.join(tokens).replace(' ##', '').strip()
    if '@' in tokens:
        out_string = out_string.replace(' ', '')
    return out_string

In [0]:
tokens

In [0]:
bert_convert_tokens_to_string(tokens)

'[CLS] who is martin luther king ? [SEP] martin luther king jr . was an american christian minister and activist who became the most visible spokesperson and leader in the civil rights movement from 1955 until his assassination in 1968 . born in atlanta , georgia , king is best known for advancing civil rights through nonviolence and civil disobedience , inspired by his christian beliefs and the nonviolent activism of mahatma gandhi . king led the 1955 montgomery bus boycott and in 1957 became the first president of the southern christian leadership conference ( sclc ) . with the sclc , he led an unsuccessful 1962 struggle against segregation in albany , georgia , and helped organize the nonviolent 1963 protests in birmingham , alabama . he helped organize the 1963 march on washington , where he delivered his famous " i have a dream " speech .'

In [0]:
tokenizer.convert_tokens_to_ids(tokens)

In [0]:
torch.tensor(tokenizer.convert_tokens_to_ids(tokens)).unsqueeze(0)

tensor([[  101,  1150,  1110, 12477,  3740,  1394,   181, 15796,  1200,  2226,
           136,   102, 12477,  3740,  1394,   181, 15796,  1200,  2226,   179,
          1197,   119,  1108,  1126,  1821, 26237,  1389, 22572, 12937,  1811,
          3907,  1105,  7041,  1150,  1245,  1103,  1211,  5085, 18391,  1105,
          2301,  1107,  1103,  2987,  2266,  2230,  1121,  3115,  1235,  1117,
         10395,  1107,  2477,   119,  1255,  1107,  1120,  9180,  1161,   117,
           176, 25690,  9037,   117,  2226,  1110,  1436,  1227,  1111, 11120,
          2987,  2266,  1194,  1664, 17417,  7836,  2093,  1105,  2987,  4267,
          7301, 18100,   117,  3768,  1118,  1117, 22572, 12937,  1811,  8810,
          1105,  1103,  1664, 17417, 13147, 18945,  1104, 12477, 11220,  1918,
           176,  5709,  3031,   119,  2226,  1521,  1103,  3115, 19863,  1204,
          2758,  4027,  1183,  3592, 21423,  1105,  1107,  3034,  1245,  1103,
          1148,  2084,  1104,  1103,  2359, 22572, 1

In [0]:
len(tokens_b),len(tokens_a)

(190, 10)

In [0]:
len(tokens)

202

In [0]:
model.eval()
with torch.no_grad():
    span_start,span_end= model(input_ids)
    

In [0]:
span_start

tensor([[-3.0330, -6.0242, -6.9201, -6.4168, -8.7065, -8.7110, -8.4517, -9.2311,
         -8.9629, -7.4666, -5.7319, -5.8057,  2.8824, -4.2409, -4.7454, -2.2637,
         -5.0260, -5.0597,  0.6874,  5.4085, -2.0698, -2.0541,  3.7607,  6.1215,
          4.2154, -1.8216, -1.7151,  2.1633, -1.2228, -1.9775,  2.8837, -3.0268,
          0.6732, -3.2847, -2.6450, -2.5519, -2.8807, -3.8289, -1.9853, -5.1151,
         -2.0583, -5.2610, -2.1343, -2.2465, -5.3153, -4.4433, -5.8708, -2.4053,
         -6.5772, -3.2967, -2.3991, -5.5036, -2.1608, -3.6182, -0.3259, -5.3628,
         -2.7122, -7.2459, -6.9572, -6.7593, -2.6473, -6.1814, -5.3302, -4.3319,
         -1.7098, -6.0527, -4.6356, -6.9700, -7.7364, -3.9609, -6.2418, -7.4918,
         -7.7684, -5.3424, -7.5105, -8.3465, -8.2392, -7.8884, -6.6476, -7.4981,
         -8.0787, -6.8077, -6.9893, -4.1639, -8.1352, -5.6401, -4.7722, -7.7572,
         -7.9954, -5.8458, -8.1577, -6.8847, -4.5623, -8.0768, -7.9936, -5.9662,
         -7.2882, -3.4128, -

In [0]:
torch.argmax(span_start),torch.argmax(span_end)

(tensor(23), tensor(32))

In [0]:
tokens[torch.argmax(span_start)]

'an'

In [0]:
tokens[-2]

'speech'

In [0]:
tokens[torch.argmax(span_end)]

'activist'

In [0]:
answer = tokens[torch.argmax(span_start):torch.argmax(span_end)+1]
answer

['an',
 'am',
 '##eric',
 '##an',
 'ch',
 '##rist',
 '##ian',
 'minister',
 'and',
 'activist']

In [0]:
answer = bert_convert_tokens_to_string(answer)
answer

'an american christian minister and activist'

##Week2. Train a Chinese Model for QA

The dataset used for pretraining

[A Span-Extraction Dataset for Chinese Machine Reading Comprehension (CMRC 2018)](https://github.com/ymcui/cmrc2018)

### Test model on Chinese text data with Chinese RoBERTa model

In [0]:
#adapt QA class to Chinese Model
class QuestionAnswering_Chinese(object):
    def __init__(self, model_path,config_file, weight_file, tokenizer_file, model_type ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.config_class, self.model_class, self.tokenizer_class = MODEL_CLASSES[model_type]
        self.config = self.config_class.from_json_file(config_file)
        # self.model = self.model_class(self.config)
        self.model = BertForQuestionAnswering.from_pretrained(path_chinese)
        # self.model.load_state_dict(torch.load(weight_file, map_location=self.device))
        self.tokenizer = self.tokenizer_class(tokenizer_file)
        self.model_type = model_type
    
    def to_list(self, tensor):
        return tensor.detach().cpu().tolist()

    def get_reply(self, question, passage):
        self.model.eval()
        with torch.no_grad():
            input_ids, _ , tokens = self.prepare_features(question, passage)
            if self.model_type == 'bert':
                span_start,span_end= self.model(input_ids)
                if (torch.argmax(span_start) <torch.argmax(span_end)):
                  answer = tokens[torch.argmax(span_start):torch.argmax(span_end)+1]
                else:
                  answer = tokens[torch.argmax(span_end):torch.argmax(span_start)+1]
                answer = self.bert_convert_tokens_to_string(answer)
            elif self.model_type == 'xlnet':
                input_vector = {'input_ids': input_ids,
                                'start_positions': None,
                                'end_positions': None }
                outputs = self.model(**input_vector)
                answer = tokens[self.to_list(outputs[1])[0][torch.argmax(outputs[0])]:self.to_list(outputs[3])[0][torch.argmax(outputs[2])]+1]
                answer = self.xlnet_convert_tokens_to_string(answer)
        return answer
    
    def bert_convert_tokens_to_string(self, tokens):
        out_string = ' '.join(tokens).replace(' ##', '').strip()
        if '@' in tokens:
            out_string = out_string.replace(' ', '')
        return out_string

    def xlnet_convert_tokens_to_string(self, tokens):
        out_string = ''.join(tokens).replace('▁', ' ').strip()
        return out_string

    def prepare_features(self, question,  passage, max_seq_length = 300, 
                 zero_pad = False, include_CLS_token = True, include_SEP_token = True):
        ## Tokenzine Input
        tokens_a = self.tokenizer.tokenize(question)
        tokens_b = self.tokenizer.tokenize(passage)
        ## Truncate
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]
        ## Initialize Tokens
        tokens = []
        if include_CLS_token:
            tokens.append(self.tokenizer.cls_token)
        ## Add Tokens and separators
        for token in tokens_a:
            tokens.append(token)
        if include_SEP_token:
            tokens.append(self.tokenizer.sep_token)
        for token in tokens_b:
            tokens.append(token)
        ## Convert Tokens to IDs
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        ## Input Mask 
        input_mask = [1] * len(input_ids)
        ## Zero-pad sequence lenght
        if zero_pad:
            while len(input_ids) < max_seq_length:
                input_ids.append(0)
                input_mask.append(0)
        return torch.tensor(input_ids).unsqueeze(0), input_mask, tokens

In [0]:
path_chinese = Path('/content/gdrive/My Drive/fastai/models_process/Roberta_wwm/')

In [0]:
bert_chinese = QuestionAnswering_Chinese(model_path=path_chinese,
    config_file =   '/content/gdrive/My Drive/fastai/models_process/Roberta_wwm/config.json',
    weight_file=    '/content/gdrive/My Drive/fastai/models_process/Roberta_wwm/pytorch_model.bin',
    tokenizer_file= '/content/gdrive/My Drive/fastai/models_process/Roberta_wwm/vocab.txt',
    model_type =    'bert'
)

In [0]:
random_text = '庞氏骗局是对金融领域投资诈骗的称呼，是金字塔骗局（Pyramid scheme）的始祖。\
很多非法的传销集团就是用这一招聚敛钱财的，这种骗术是一个名叫查尔斯·庞兹的投机商人“发明”的。\
庞氏骗局在中国又称“拆东墙补西墙”“空手套白狼”。简言之就是利用新投资人的钱来向老投资者支付利息和短期回报，以制造赚钱的假象进而骗取更多的投资。'

In [0]:
question1 = '什么是庞氏骗局？'

In [0]:
bert_chinese.get_reply(question1,random_text)

'氏 骗 局 是 对 金 融 领 域 投 资 诈 骗 的 称 呼 ， 是 金 字 塔 骗 局 （ pyramid scheme ） 的 始 祖 。 很 多 非 法 的 传 销 集 团 就 是 用 这 一 招 聚 敛 钱 财 的 ， 这 种 骗 术 是 一 个 名 叫 查 尔 斯 · 庞 兹 的 投 机 商 人 [UNK] 发 明'

In [0]:
tokenizer = BertTokenizer.from_pretrained(path_chinese)
bert_model = BertForQuestionAnswering.from_pretrained(path_chinese)

In [0]:
config_file =   'bert-large-cased-whole-word-masking-finetuned-squad-config.json'
weight_file=    'bert-large-cased-whole-word-masking-finetuned-squad-pytorch_model.bin'
tokenizer_file= 'bert-large-cased-whole-word-masking-finetuned-squad-vocab.txt'
model_type =    'bert'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]
config = config_class.from_json_file(config_file)
model = model_class(config)
model.load_state_dict(torch.load(weight_file, map_location=device))
tokenizer = tokenizer_class(tokenizer_file)

In [0]:
tokens_a = tokenizer.tokenize(question1)
tokens_b = tokenizer.tokenize(random_text)

In [0]:
tokens_a

['什', '么', '是', '庞', '氏', '骗', '局', '？']

In [0]:
tokens_b

['庞',
 '氏',
 '骗',
 '局',
 '是',
 '对',
 '金',
 '融',
 '领',
 '域',
 '投',
 '资',
 '诈',
 '骗',
 '的',
 '称',
 '呼',
 '，',
 '是',
 '金',
 '字',
 '塔',
 '骗',
 '局',
 '（',
 'p',
 '##y',
 '##ram',
 '##id',
 'sc',
 '##he',
 '##me',
 '）',
 '的',
 '始',
 '祖',
 '。',
 '很',
 '多',
 '非',
 '法',
 '的',
 '传',
 '销',
 '集',
 '团',
 '就',
 '是',
 '用',
 '这',
 '一',
 '招',
 '聚',
 '敛',
 '钱',
 '财',
 '的',
 '，',
 '这',
 '种',
 '骗',
 '术',
 '是',
 '一',
 '个',
 '名',
 '叫',
 '查',
 '尔',
 '斯',
 '·',
 '庞',
 '兹',
 '的',
 '投',
 '机',
 '商',
 '人',
 '[UNK]',
 '发',
 '明',
 '[UNK]',
 '的',
 '。',
 '庞',
 '氏',
 '骗',
 '局',
 '在',
 '中',
 '国',
 '又',
 '称',
 '[UNK]',
 '拆',
 '东',
 '墙',
 '补',
 '西',
 '墙',
 '[UNK]',
 '[UNK]',
 '空',
 '手',
 '套',
 '白',
 '狼',
 '[UNK]',
 '。',
 '简',
 '言',
 '之',
 '就',
 '是',
 '利',
 '用',
 '新',
 '投',
 '资',
 '人',
 '的',
 '钱',
 '来',
 '向',
 '老',
 '投',
 '资',
 '者',
 '支',
 '付',
 '利',
 '息',
 '和',
 '短',
 '期',
 '回',
 '报',
 '，',
 '以',
 '制',
 '造',
 '赚',
 '钱',
 '的',
 '假',
 '象',
 '进',
 '而',
 '骗',
 '取',
 '更',
 '多',
 '的',
 '投',
 '资',
 '。']

In [0]:
max_seq_length = 300
zero_pad = False
include_CLS_token = True
include_SEP_token = True

In [0]:
if len(tokens_a) > max_seq_length - 2:
  tokens_a = tokens_a[0:(max_seq_length - 2)]
  ## Initialize Tokens
tokens = []
if include_CLS_token:
    tokens.append(tokenizer.cls_token)
## Add Tokens and separators
for token in tokens_a:
    tokens.append(token)
if include_SEP_token:
    tokens.append(tokenizer.sep_token)
for token in tokens_b:
    tokens.append(token)
## Convert Tokens to IDs
input_ids = tokenizer.convert_tokens_to_ids(tokens)
## Zero-pad sequence length
if zero_pad:
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
input_ids=torch.tensor(input_ids).unsqueeze(0)


In [0]:
def bert_convert_tokens_to_string(tokens):
    out_string = ' '.join(tokens).replace(' ##', '').strip()
    if '@' in tokens:
        out_string = out_string.replace(' ', '')
    return out_string

In [0]:
tokens

In [0]:
bert_convert_tokens_to_string(tokens)

'[CLS] 什 么 是 庞 氏 骗 局 ？ [SEP] 庞 氏 骗 局 是 对 金 融 领 域 投 资 诈 骗 的 称 呼 ， 是 金 字 塔 骗 局 （ pyramid scheme ） 的 始 祖 。 很 多 非 法 的 传 销 集 团 就 是 用 这 一 招 聚 敛 钱 财 的 ， 这 种 骗 术 是 一 个 名 叫 查 尔 斯 · 庞 兹 的 投 机 商 人 [UNK] 发 明 [UNK] 的 。 庞 氏 骗 局 在 中 国 又 称 [UNK] 拆 东 墙 补 西 墙 [UNK] [UNK] 空 手 套 白 狼 [UNK] 。 简 言 之 就 是 利 用 新 投 资 人 的 钱 来 向 老 投 资 者 支 付 利 息 和 短 期 回 报 ， 以 制 造 赚 钱 的 假 象 进 而 骗 取 更 多 的 投 资 。'

In [0]:
tokenizer.convert_tokens_to_ids(tokens)

In [0]:
torch.tensor(tokenizer.convert_tokens_to_ids(tokens)).unsqueeze(0)

tensor([[  101,   784,   720,  3221,  2425,  3694,  7745,  2229,  8043,   102,
          2425,  3694,  7745,  2229,  3221,  2190,  7032,  6084,  7566,  1818,
          2832,  6598,  6400,  7745,  4638,  4917,  1461,  8024,  3221,  7032,
          2099,  1849,  7745,  2229,  8020,   158,  8179,  9661,  8601, 11515,
          9977,  8505,  8021,  4638,  1993,  4862,   511,  2523,  1914,  7478,
          3791,  4638,   837,  7218,  7415,  1730,  2218,  3221,  4500,  6821,
           671,  2875,  5471,  3137,  7178,  6568,  4638,  8024,  6821,  4905,
          7745,  3318,  3221,   671,   702,  1399,  1373,  3389,  2209,  3172,
           185,  2425,  1074,  4638,  2832,  3322,  1555,   782,   100,  1355,
          3209,   100,  4638,   511,  2425,  3694,  7745,  2229,  1762,   704,
          1744,  1348,  4917,   100,  2858,   691,  1870,  6133,  6205,  1870,
           100,   100,  4958,  2797,  1947,  4635,  4331,   100,   511,  5042,
          6241,   722,  2218,  3221,  1164,  4500,  

In [0]:
len(tokens_b),len(tokens_a)

(156, 8)

In [0]:
len(tokens)

166

In [0]:
model.eval()
with torch.no_grad():
    span_start,span_end= model(input_ids)
    

In [0]:
span_start

tensor([[-1.1809, -4.9494, -7.2212, -7.5759, -7.6350, -8.3451, -9.2880, -8.4027,
         -8.5443, -7.9771, -4.2551, -5.5678, -8.3149, -6.4926, -2.9097, -4.5758,
         -3.4842, -2.6518, -3.9101, -6.2560, -1.4267, -5.6368, -2.7000, -6.7558,
         -6.2690, -3.4655, -4.1038, -4.2167, -3.3466, -4.2871, -4.2026, -4.4519,
         -7.4884, -5.7710, -3.0653, -4.4859, -5.3675, -4.4587, -4.5967, -5.7832,
         -5.4509, -5.0483, -5.2581, -7.3320, -5.2307, -7.9581, -7.5924, -5.7384,
         -5.7428, -6.6199, -5.6846, -7.2540, -5.1651, -4.6019, -4.8054, -5.7441,
         -4.8727, -5.7000, -4.7641, -5.1740, -4.8043, -7.8425, -5.8914, -4.8307,
         -5.5440, -6.0415, -7.1576, -4.7195, -4.8563, -4.7727, -7.0551, -4.8793,
         -5.3432, -4.9334, -5.5517, -7.4773, -6.2997, -5.1078, -5.8286, -6.0578,
         -5.6377, -5.1083, -4.3592, -7.2845, -2.1229, -3.1430, -4.0567, -6.1290,
         -3.9296, -4.8173, -4.8749, -4.6231, -6.4750, -7.7132, -4.1228, -5.0808,
         -8.2383, -5.6928, -

In [0]:
chinese_facts = '"尼尔·奥尔登·阿姆斯特朗（Neil Alden Armstrong，1930年8月5日－2012年8月25日），\
美国宇航员、试飞员、海军飞行员以及大学教授。\
在美国国家航空航天局服役时，阿姆斯特朗于1969年7月21日时成为了第一个踏上月球的宇航员，\
也是第一个在地球外星体上留下脚印的人类成员，而其搭档巴兹·奥尔德林也成为了第二位登上月球后安全返回的人，\
两人在月球表面停留了两个半小时。阿姆斯特朗的首次太空任务是双子星座8号，在这次任务中，他和大卫·斯科特执行了历史上第一次轨道对接。\
1969年7月，阿姆斯特朗在执行他的第二次也是最后一次太空任务阿波罗11号时，迈出了“人类的一大步”。 \
2012年8月25日，因心脏搭桥手术后的并发症逝世，享年82岁。其家人在一份声明中称，阿姆斯特朗死于8月初心脏搭桥手术后的并发症。\
美国总统奥巴马27日下令，全国将在首位成功登月的宇航员阿姆斯特朗葬礼举行之日全天降半旗，向这位传奇人物致敬、寄托哀思。\
阿姆斯特朗逝世消息公布之后，美国及世界各地民众也通过各种方式缅怀这位登月先驱。\
美国航天局月球科学研究所则通过网络呼吁大众“对月亮眨眨眼睛”向他致敬"'

chinese_questions = [
                     '谁是尼尔·奥尔登·阿姆斯特朗?',
                     '阿姆斯特朗的搭档是谁?',
                     '是谁下令在27日向阿姆斯特朗致敬?'
]



In [0]:
bert_chinese.get_reply(chinese_questions[2],chinese_facts)

'， 美 国 宇 航 员 、 试 飞 员 、 海 军 飞 行 员 以 及 大 学 教 授 。 在 美 国 国 家 航 空 航 天 局 服 役 时 ， 阿 姆 斯 特 朗 于 1969 年 7 月 21 日 时 成 为 了 第 一 个 踏 上 月 球 的 宇 航 员 ， 也 是 第 一 个 在 地 球 外 星 体 上 留 下 脚 印 的 人 类 成 员 ， 而 其 搭 档 巴 兹 · 奥 尔 德 林 也 成 为 了 第 二 位 登 上 月 球 后 安 全 返 回 的 人 ， 两 人 在 月 球 表 面 停 留 了 两 个 半 小 时 。 阿 姆 斯 特 朗 的 首 次 太 空 任 务 是 双 子 星 座 8 号 ， 在 这 次 任 务 中 ， 他 和 大 卫 · 斯 科 特 执 行 了 历 史 上 第 一 次 轨 道 对 接 。 1969 年 7 月 ， 阿 姆 斯 特 朗 在 执 行 他 的 第 二 次 也 是 最 后 一 次 太 空 任 务 阿 波 罗 11 号 时 ， 迈 出 了 [UNK] 人 类 的 一 大 步 [UNK] 。 2012 年 8 月 25 日 ， 因 心 脏 搭 桥 手 术 后 的 并 发 症 逝 世 ， 享 年 82 岁 。 其 家 人 在 一 份 声 明 中 称 ， 阿 姆 斯 特 朗 死 于 8 月 初 心 脏 搭 桥 手 术 后 的 并 发 症 。 美 国 总 统 奥 巴 马 27 日 下 令 ， 全 国 将 在 首 位 成 功 登 月 的 宇 航 员 阿 姆 斯 特 朗 葬 礼 举 行 之 日 全 天 降 半 旗 ， 向 这 位 传 奇'

In [0]:
chinese_facts = '《战国无双3》是由光荣和ω-force开发的战国无双系列的正统第三续作。\
本作以三大故事为主轴，分别是以武田信玄等人为主的《关东三国志》，织田信长等人为主的《战国三杰》，石田三成等人为主的《关原的年轻武者》，丰富游戏内的剧情。\
此部份专门介绍角色，欲知武器情报、奥义字或擅长攻击类型等，请至战国无双系列1.由于乡里大辅先生因故去世，不得不寻找其他声优接手。从猛将传 and Z开始。\
2.战国无双 编年史的原创男女主角亦有专属声优。此模式是任天堂游戏谜之村雨城改编的新增模式。本作中共有20张战场地图（不含村雨城），后来发行的猛将传再新增3张战场地图。\
但游戏内战役数量繁多，部分地图会有兼用的状况，战役虚实则是以光荣发行的2本「战国无双3 人物真书」内容为主，以下是相关介绍。'

chinese_questions = [
                     '《战国无双3》是由哪两个公司合作开发的？',
                     '战国史模式主打哪两个模式？'
]


In [0]:
bert_chinese.get_reply(chinese_questions[0],chinese_facts)

'主 轴 ， 分 别 是 以 武 田 信 玄 等 人 为 主 的 《 关 东 三 国 志 》 ， 织 田 信 长 等 人 为 主 的 《 战 国 三 杰 》 ， 石 田 三 成 等 人 为 主 的 《 关 原 的 年 轻 武 者 》 ， 丰 富 游 戏 内 的 剧 情 。 此 部 份 专 门 介 绍 角 色 ， 欲 知 武 器 情 报 、 奥 义'

##Week3. Fine-tune the Model on downstream Chinese text data

##Final Week. Interactive APP for QA Demo on Chinese text data